In [16]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
subgraph_llm = ChatOpenAI(model='gpt-4o')
parent_llm = ChatOpenAI(model='gpt-4o-mini')

In [17]:
class SubState(TypedDict):

    input_text: str
    translated_text: str

In [18]:
def translate_text(state: SubState):

    prompt = f"""
Translate the following text to Bengali.
Keep it natural and clear. Do not add extra content.

Text:
{state["input_text"]}
""".strip()
    
    translated_text = subgraph_llm.invoke(prompt).content

    return {'translated_text': translated_text}

In [19]:
subgraph_builder = StateGraph(SubState)

subgraph_builder.add_node('translate_text', translate_text)

subgraph_builder.add_edge(START, 'translate_text')
subgraph_builder.add_edge('translate_text', END)

subgraph = subgraph_builder.compile()

In [20]:
class ParentState(TypedDict):

    question: str
    answer_eng: str
    answer_hin: str

In [21]:
def generate_answer(state: ParentState):

    answer = parent_llm.invoke(f"You are a helpful assistant. Answer clearly.\n\nQuestion: {state['question']}").content
    return {'answer_eng': answer}

In [22]:
def translate_answer(state: ParentState):

    # call the subgraph
    result = subgraph.invoke({'input_text': state['answer_eng']})

    return {'answer_hin': result['translated_text']}

In [23]:
parent_builder = StateGraph(ParentState)

parent_builder.add_node("answer", generate_answer)
parent_builder.add_node("translate", translate_answer)

parent_builder.add_edge(START, 'answer')
parent_builder.add_edge('answer', 'translate')
parent_builder.add_edge('translate', END)

graph = parent_builder.compile()

In [24]:
graph.invoke({'question': 'What is quantum physics'})

{'question': 'What is quantum physics',
 'answer_eng': "Quantum physics, or quantum mechanics, is a fundamental branch of physics that describes the physical properties of nature at the scale of atoms and subatomic particles. It departs from classical mechanics and introduces concepts such as quantization, wave-particle duality, the uncertainty principle, and superposition.\n\nKey features of quantum physics include:\n\n1. **Wave-Particle Duality**: Particles, such as electrons and photons, exhibit both wave-like and particle-like behavior depending on how they are observed.\n\n2. **Quantization**: Energy levels in a quantum system are discrete rather than continuous. For example, electrons can only occupy specific energy levels in an atom.\n\n3. **Uncertainty Principle**: Formulated by Werner Heisenberg, it states that certain pairs of physical properties, like position and momentum, cannot be simultaneously measured with arbitrary precision.\n\n4. **Superposition**: A quantum system 